##### Read raw files

In [ ]:
import pandas as pd
import os
from sas7bdat import SAS7BDAT

# Define the base data directory
base_data_dir = ""

# Function to read SAS file
def read_sas_file(file_path):
    with SAS7BDAT(file_path) as file:
        return file.to_data_frame()

# Read Analysis Data Sets
analysis_files = {
    'accord_key': 'accord_key.sas7bdat',
    'activitystatus': 'activitystatus.sas7bdat',
    'bloodpressure': 'bloodpressure.sas7bdat',
    'concomitantmeds': 'concomitantmeds.sas7bdat',
    'cvdoutcomes': 'cvdoutcomes.sas7bdat',
    'eye': 'eye.sas7bdat',
    'hba1c': 'hba1c.sas7bdat',
    'hypoglycemiaevents': 'hypoglycemiaevents.sas7bdat',
    'hypoglycemiatime1st': 'hypoglycemiatime1st.sas7bdat',
    'lipids': 'lipids.sas7bdat',
    'microvascularoutcomes': 'microvascularoutcomes.sas7bdat',
    'otherlabs': 'otherlabs.sas7bdat',
    'sae': 'sae.sas7bdat'
}

# Read CRF Data Sets
crf_files = {
    'baseline': 'f07_baselinehistoryphysicalexam.sas7bdat',
    'anti_hyperglycemic': 'f10_glycemiamedicationslog.sas7bdat',
    'annual_followup': 'f36_annualfollowupphysicalexam.sas7bdat'
}

# Read Analysis Data Sets
for var_name, file_name in analysis_files.items():
    file_path = os.path.join(base_data_dir, 'accord', '3a-Analysis_Data_Sets', file_name)
    print(f"Reading {file_name}...")
    globals()[var_name] = read_sas_file(file_path)
    print(f"Shape of {var_name}: {globals()[var_name].shape}")

# Read CRF Data Sets
for var_name, file_name in crf_files.items():
    file_path = os.path.join(base_data_dir, 'accord', '4a-CRF_Data_Sets', file_name)
    print(f"Reading {file_name}...")
    globals()[var_name] = read_sas_file(file_path)
    print(f"Shape of {var_name}: {globals()[var_name].shape}")

# Print summary of all loaded datasets
print("\nSummary of loaded datasets:")
for dataset_name in list(analysis_files.keys()) + list(crf_files.keys()):
    df = globals()[dataset_name]
    print(f"\n{dataset_name}:")
    print(f"  Shape: {df.shape}")
    print(f"  Columns: {df.columns.tolist()[:5]}...")

##### Get baseline characteristics status from baseline dataset

In [ ]:
# Create baseline characteristics
baseline_char = accord_key[['MaskID', 'female', 'baseline_age', 'raceclass', 'cvd_hx_baseline']].copy()
baseline_char.insert(1, 'Visit', 'BLR')  # Insert Visit column after MaskID
baseline_char['black'] = (baseline_char['raceclass'] == 'Black').astype(int)  # Create binary black indicator
baseline_char = baseline_char.drop('raceclass', axis=1)  # Remove original raceclass column

# Create one-year characteristics
char_2_year = baseline_char.copy()
char_2_year['Visit'] = 'F24'  # Change visit to F24
char_2_year['baseline_age'] = char_2_year['baseline_age'] + 2  # Add 2 to age

# Combine both dataframes
characteristics = pd.concat([baseline_char, char_2_year], ignore_index=True)

# Display information about the processed data
print("Baseline Characteristics Shape:", baseline_char.shape)
print("\nFirst few rows of baseline characteristics:")
print(baseline_char.head())

print("\nOne-Year Characteristics Shape:", char_2_year.shape)
print("\nFirst few rows of one-year characteristics:")
print(char_2_year.head())

print("\nCombined Characteristics Shape:", characteristics.shape)
print("\nFirst few rows of combined characteristics:")
print(characteristics.head(10))  # Show first 10 rows to see both BLR and F24 records

# Show summary statistics
print("\nSummary of characteristics by visit:")
for visit in ['BLR', 'F24']:
    subset = characteristics[characteristics['Visit'] == visit]
    print(f"\n{visit} Visit Summary:")
    print(f"Number of patients: {len(subset)}")
    print(f"Average age: {subset['baseline_age'].mean():.2f}")
    print(f"Percentage female: {(subset['female'].mean() * 100):.1f}%")
    print(f"Percentage black: {(subset['black'].mean() * 100):.1f}%")
    print(f"Percentage with CVD history: {(subset['cvd_hx_baseline'].mean() * 100):.1f}%")

##### Get smoking status from baseline dataset and annual follow up

In [ ]:
# Get smoking status and BMI from baseline
baseline_smoke_bmi = baseline[['MaskID', 'Visit', 'cigarett', 'wt_kg', 'ht_cm']].copy()
baseline_smoke_bmi.columns = ['MaskID', 'Visit', 'smoke', 'weight', 'height']

# Convert cigarett values to binary smoke status
# 1 -> 1 (smoker)
# 2 -> 0 (non-smoker)
baseline_smoke_bmi = baseline_smoke_bmi[baseline_smoke_bmi['smoke'].isin([1, 2])]
baseline_smoke_bmi['smoke'] = (baseline_smoke_bmi['smoke'] == 1).astype(int)

# Calculate BMI for baseline
baseline_smoke_bmi['bmi'] = baseline_smoke_bmi['weight'] / ((baseline_smoke_bmi['height']/100) ** 2)

# Display information about baseline data
print("Baseline Dataset shape:", baseline_smoke_bmi.shape)
print("\nBaseline smoking status counts:")
smoke_counts = baseline_smoke_bmi['smoke'].value_counts()
print(smoke_counts)
print("\nBaseline BMI summary:")
print(baseline_smoke_bmi['bmi'].describe())

# Get smoking status and BMI from annual followup
followup_smoke_bmi = annual_followup[['MaskID', 'Visit', 'smoked', 'wt_kg', 'ht_cm']].copy()
followup_smoke_bmi.columns = ['MaskID', 'Visit', 'smoke', 'weight', 'height']

# Keep only F24 visits
followup_smoke_bmi = followup_smoke_bmi[followup_smoke_bmi['Visit'] == 'F24']

# Convert smoked values to binary smoke status
# 1 -> 1 (smoker)
# 2 -> 0 (non-smoker)
followup_smoke_bmi = followup_smoke_bmi[followup_smoke_bmi['smoke'].isin([1, 2])]
followup_smoke_bmi['smoke'] = (followup_smoke_bmi['smoke'] == 1).astype(int)

# Calculate BMI for follow-up
followup_smoke_bmi['bmi'] = followup_smoke_bmi['weight'] / ((followup_smoke_bmi['height']/100) ** 2)

# Display information about follow-up data
print("\nFollow-up Dataset (F24 visits only):")
print("Shape:", followup_smoke_bmi.shape)
print("\nSmoking status counts at F24:")
smoke_counts = followup_smoke_bmi['smoke'].value_counts()
print(smoke_counts)
print("\nF24 BMI summary:")
print(followup_smoke_bmi['bmi'].describe())

# Combine baseline and follow-up data
smoke_bmi = pd.concat([baseline_smoke_bmi, followup_smoke_bmi], ignore_index=True)

# Drop the intermediate height and weight columns
smoke_bmi = smoke_bmi[['MaskID', 'Visit', 'smoke', 'bmi']]

print("\nCombined dataset summary by visit:")
for visit in ['BLR', 'F24']:
    subset = smoke_bmi[smoke_bmi['Visit'] == visit]
    print(f"\n{visit} Visit:")
    print(f"Number of patients: {len(subset)}")
    print(f"Smoking percentage: {(subset['smoke'].mean() * 100):.1f}%")
    print(f"Average BMI: {subset['bmi'].mean():.1f}")
    print(f"Median BMI: {subset['bmi'].median():.1f}")

print("\nFirst few rows of final dataset:")
print(smoke_bmi.head())

In [ ]:
# Get total number of unique patients
total_patients = activitystatus['MaskID'].nunique()

# Get EXIT visit information
exit_visits = activitystatus[activitystatus['Visit'] == 'EXIT']
patients_with_exit = exit_visits['MaskID'].nunique()

# Calculate percentage
exit_percentage = (patients_with_exit / total_patients) * 100

print(f"Total unique patients: {total_patients}")
print(f"Patients with EXIT visit: {patients_with_exit}")
print(f"Percentage of patients with EXIT visit: {exit_percentage:.2f}%")

# Show first few rows of EXIT visits for reference
print("\nSample of EXIT visit records:")
print(exit_visits[['MaskID', 'Visit']].head())

In [ ]:
# Create heart_function variable with blood pressure measurements
heart_function = bloodpressure[['MaskID', 'Visit', 'sbp', 'dbp', 'hr']].copy()

# Keep only BLR and F24 visits
heart_function = heart_function[heart_function['Visit'].isin(['BLR', 'F24'])]

# Display information about the processed data
print("Dataset shape:", heart_function.shape)
print("\nNumber of records by visit:")
print(heart_function['Visit'].value_counts())
print("\nFirst few rows:")
print(heart_function.head())

In [ ]:
hba1c_measurements = hba1c[['MaskID', 'Visit', 'hba1c']].copy()

# Keep only BLR and F24 visits
hba1c_measurements = hba1c_measurements[hba1c_measurements['Visit'].isin(['BLR', 'F24'])]

# Display information about the processed data
print("Dataset shape:", hba1c_measurements.shape)
print("\nNumber of records by visit:")
print(hba1c_measurements['Visit'].value_counts())
print("\nFirst few rows:")
print(hba1c_measurements.head())

In [ ]:
lipids_measurements = lipids[['MaskID', 'Visit', 'chol', 'ldl', 'hdl', 'trig']].copy()

# Keep only BLR and F24 visits
lipids_measurements = lipids_measurements[lipids_measurements['Visit'].isin(['BLR', 'F24'])]

# Display information about the processed data
print("Dataset shape:", lipids_measurements.shape)
print("\nNumber of records by visit:")
print(lipids_measurements['Visit'].value_counts())
print("\nFirst few rows:")
print(lipids_measurements.head())

In [ ]:
otherlabs_measurements = otherlabs[['MaskID', 'Visit','fpg' ,'potassium','screat','uacr']].copy()

# Keep only BLR and F24 visits
otherlabs_measurements = otherlabs_measurements[otherlabs_measurements['Visit'].isin(['BLR', 'F24'])]

# Display information about the processed data
print("Dataset shape:", otherlabs_measurements.shape)
print("\nNumber of records by visit:")
print(otherlabs_measurements['Visit'].value_counts())
print("\nFirst few rows:")
print(otherlabs_measurements.head())

In [ ]:
# Add bprx column based on any BP medication
concomitantmeds['bprx'] = (
    (concomitantmeds['thiazide'] == 1) |
    (concomitantmeds['ksparing'] == 1) |
    (concomitantmeds['loop'] == 1) |
    (concomitantmeds['a2rb'] == 1) |
    (concomitantmeds['acei'] == 1) |
    (concomitantmeds['dhp_ccb'] == 1) |
    (concomitantmeds['nondhp_ccb'] == 1) |
    (concomitantmeds['alpha_blocker'] == 1) |
    (concomitantmeds['central_agent'] == 1) |
    (concomitantmeds['beta_blocker'] == 1) |
    (concomitantmeds['vasodilator'] == 1) |
    (concomitantmeds['reserpine'] == 1) |
    (concomitantmeds['other_bpmed'] == 1) |
    (concomitantmeds['nitrate'] == 1)
).astype(int)  # Convert boolean to 0/1

# Ensure statin is binary 0/1
concomitantmeds['statin'] = (concomitantmeds['statin'] == 1).astype(int)

# Select specific columns and filter visits
med_subset = concomitantmeds[['MaskID', 'Visit', 'bprx', 'statin']].copy()
med_subset = med_subset[med_subset['Visit'].isin(['BLR', 'F24'])]

# Display information about the processed data
print("Dataset shape:", med_subset.shape)
print("\nNumber of records by visit:")
print(med_subset['Visit'].value_counts())
print("\nMedication usage summary by visit:")
for visit in ['BLR', 'F24']:
    subset = med_subset[med_subset['Visit'] == visit]
    print(f"\n{visit} Visit:")
    print(f"BP medication usage: {(subset['bprx'].mean() * 100):.1f}%")
    print(f"Statin usage: {(subset['statin'].mean() * 100):.1f}%")
print("\nFirst few rows:")
print(med_subset.head())

In [ ]:
# Merge all datasets based on MaskID and Visit
merged_data = characteristics.merge(
    smoke_bmi, on=['MaskID', 'Visit'], how='left'
).merge(
    heart_function, on=['MaskID', 'Visit'], how='left'
).merge(
    hba1c_measurements, on=['MaskID', 'Visit'], how='left'
).merge(
    lipids_measurements, on=['MaskID', 'Visit'], how='left'
).merge(
    otherlabs_measurements, on=['MaskID', 'Visit'], how='left'
).merge(
    med_subset, on=['MaskID', 'Visit'], how='left'
)

# Display information about the merged dataset
print("Merged dataset shape:", merged_data.shape)
print("\nNumber of records by visit:")
print(merged_data['Visit'].value_counts())

# Show number of non-null values for each column
print("\nNumber of non-null values in each column:")
print(merged_data.count())

# Calculate percentage of missing values for each column
print("\nPercentage of missing values in each column:")
missing_percentages = (merged_data.isnull().sum() / len(merged_data) * 100).round(2)
for column, percentage in missing_percentages[missing_percentages > 0].items():
    print(f"{column}: {percentage}%")

# Show summary statistics for numeric columns
print("\nSummary statistics for numeric columns:")
numeric_columns = merged_data.select_dtypes(include=['float64', 'int64']).columns
print(merged_data[numeric_columns].describe().round(2))

print("\nFirst few rows of merged dataset:")
print(merged_data.head())

In [ ]:

merged_data.dropna(inplace=True)
merged_data.tail()

In [ ]:
merged_data[merged_data['Visit']=='F24']

In [ ]:
merged_data.reset_index(drop=True, inplace=True)
merged_data[merged_data['Visit']=='F24'].to_csv('data/accord_f24.csv', index=False)
merged_data[merged_data['Visit']=='BLR'].to_csv('data/accord_blr.csv', index=False)

##### outcomes

In [ ]:
# Create outcomes dataset with only po-related variables
selected_columns = ['MaskID', 'censor_po', 'fuyrs_po']
outcomes = cvdoutcomes[selected_columns].copy()


outcomes['cvd'] = (outcomes['censor_po'] == 0).astype(int)  # 1 if event occurred, 0 if censored

outcomes[['MaskID', 'cvd']].to_csv('data/accord_outcomes.csv', index=False)
